In [ ]:
import casperfpga
from casperfpga import snapadc
import logging
logger=logging.getLogger('pmc')
logging.basicConfig(filename='pmc.log',level=logging.DEBUG)
snap=casperfpga.CasperFpga('192.168.2.100',69,logger=logger)

In [ ]:
snap.upload_to_ram_and_program('fpg/dsa10_2022-11-02_1656.fpg')

In [ ]:
snap.adcs

In [ ]:
snap.is_connected()

In [ ]:
snap.listdev()

In [ ]:
snap.transport.get_metadata()

In [ ]:
snap.get_system_information(filename='fpg/dsa10m_2022-11-02_1656.fpg',initialise_objects=False)

In [ ]:
adc=snap.adcs['snap_adc']

In [ ]:
adc.selectADC()
adc.init(sample_rate=500,numChannel=2)

In [ ]:
adc.resolution

In [ ]:
gbe0=snap.gbes['eth_gbe0']
gbe1=snap.gbes['eth1_gbe1']

In [113]:
gbe0.get_gbe_core_details()


{'ip_prefix': '192.168.5',
 'ip': IpAddress(192.168.5.20),
 'subnet_mask': IpAddress(255.255.255.0),
 'mac': Mac(12:34:56:78:00:00),
 'gateway_ip': IpAddress(0.0.0.1),
 'fabric_port': 10000,
 'fabric_en': 0,
 'multicast': {'base_ip': IpAddress(255.255.255.255),
  'ip_mask': IpAddress(255.255.255.255),
  'rx_ips': []}}

In [ ]:
gbe1.get_gbe_core_details()

In [ ]:
adc.snapshot()

In [ ]:
data=adc.readRAM()

In [ ]:
data[0][1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(data[2])
plt.show()

In [112]:
src_mac  = "00:08:0b:c4:17:01"
src_ip   = "192.168.2.100"
src_port = 4000
gbe1.configure_core(src_mac, src_ip, src_port)
gbe1.get_gbe_core_details()

{'ip_prefix': '192.168.2',
 'ip': IpAddress(192.168.2.100),
 'subnet_mask': IpAddress(255.255.255.0),
 'mac': Mac(00:08:0B:C4:17:01),
 'gateway_ip': IpAddress(0.0.0.1),
 'fabric_port': 4000,
 'fabric_en': 1,
 'multicast': {'base_ip': IpAddress(255.255.255.255),
  'ip_mask': IpAddress(255.255.255.255),
  'rx_ips': []}}

In [ ]:
gbe1_dst_mac  = 0xf452141624a0
# write register requires a int vaule, but set_single_arp_entry requires a string
gbe1_dst_ip   = 192*(2**24) + 168*(2**16) + 2*(2**8) + 1
gbe1_dst_ip_str='192.168.2.1'
gbe1_dst_port = 4015
snap.registers['ip1'].write_int(gbe1_dst_ip)
snap.registers['port1'].write_int(gbe1_dst_port)
gbe1.set_single_arp_entry(gbe1_dst_ip_str,gbe1_dst_mac)
gbe1.get_arp_details()

In [ ]:
gbe1.set_single_arp_entry('192.168.2.1',0xf452141624a0)

In [ ]:
gbe1.fabric_enable()

In [ ]:
import time
# set acc len
snap.registers['acc_len'].write_int(127)
# full rst
snap.registers['force_sync'].write_int(2) 
time.sleep(0.1)
# sync
snap.registers['force_sync'].write_int(1)
time.sleep(0.1)
snap.registers['force_sync'].write_int(0)

In [ ]:
snap.registers['scaling'].write_int(0)

In [64]:
snap.registers['fft_of_count'].read_uint()

1189190492

In [59]:
snap.registers['fft_shift'].read_uint()

65535

In [58]:
snap.registers['fft_shift'].write_int(65535)

In [ ]:
snap.registers['sel1'].read_uint()

In [ ]:
snap.registers['sel1'].write_int(1)

In [ ]:

snap.registers['coeff1'].write_int(7)

In [ ]:
snap.registers['coeff1'].read_uint()

In [30]:
snap.registers['ip1'].read_uint()

3232236033

In [31]:
snap.registers['port1'].read_uint()

4015

In [65]:
snap.memory_devices

{'acc_16': Sbram:acc_16,
 'acc_full': Sbram:acc_full,
 'adc_snap_bram': Sbram:adc_snap_bram,
 'debug1_pfb': Sbram:debug1_pfb,
 'debug2_pfb': Sbram:debug2_pfb,
 'debug3_pfb': Sbram:debug3_pfb,
 'debug4_pfb': Sbram:debug4_pfb,
 'debug_eq_dat': Sbram:debug_eq_dat,
 'eq_0_coeffs': Sbram:eq_0_coeffs,
 'eq_1_coeffs': Sbram:eq_1_coeffs,
 'eq_2_coeffs': Sbram:eq_2_coeffs,
 'eq_3_coeffs': Sbram:eq_3_coeffs,
 'raw1': Sbram:raw1,
 'raw2': Sbram:raw2,
 'acc_len': <casperfpga.register.Register at 0x7f8c49fe3d60>,
 'adc_snap_ctrl': <casperfpga.register.Register at 0x7f8c49fe3e20>,
 'adc_snap_status': <casperfpga.register.Register at 0x7f8c49fe3dc0>,
 'adc_trig': <casperfpga.register.Register at 0x7f8c49fe3ee0>,
 'coeff1': <casperfpga.register.Register at 0x7f8c49fe3b80>,
 'del1': <casperfpga.register.Register at 0x7f8c4a17da30>,
 'del2': <casperfpga.register.Register at 0x7f8c49ff3eb0>,
 'del3': <casperfpga.register.Register at 0x7f8c49ff3a60>,
 'del4': <casperfpga.register.Register at 0x7f8c49ff37c

In [111]:
d=snap.memory_devices['acc_16'].read_raw()
print(len(d))
print(d[0])

2
b'\xf5B\xb0Fw\x80a\x12\xb0Fw\x80a\x12[\xb4w\x80a\x12[\xb4QCa\x12[\xb4QCN\xd0[\xb4QCN\xd0O!QCN\xd0O!V\x80N\xd0O!V\x80JDO!V\x80JDJ\x1aV\x80JDJ\x1aKMJDJ\x1aKMI\xbcJ\x1aKMI\xbcR~KMI\xbcR~HGI\xbcR~HGD\x15R~HGD\x15DrHGD\x15DrH\x81D\x15DrH\x81AMDrH\x81AMGNA\x14C\x8fG%I\xa3C\x8fG%I\xa3J\x02G%I\xa3J\x02EiI\xa3J\x02EiK`J\x02EiK`F5EiK`F5D\x13K`F5D\x13G_F5D\x13G_H\xcaD\x13G_H\xcaF\x12G_H\xcaF\x12L\x87H\xcaF\x12L\x87N\xa8F\x12L\x87N\xa8FEL\x87N\xa8FEF\x05N\xa8FEF\x05@\xf7FEF\x05@\xf7H\xbbF\x05@\xf7H\xbbHx@\xf7H\xbbHxG\xd4H\xbbHxG\xd4C\x89HxG\xd4C\x89?\xc1G\xd4C\x89?\xc1A\xf8C\x89?\xc1A\xf8B\x95?\xc1A\xf8B\x95A\xeeA\xf8B\x95A\xeeI\xabB\x95A\xeeI\xab?qA\xeeI\xab?qG\xecI\xab?qG\xecBp?qG\xecBp?7G\xecBp?7CZBp?7CZ<*?7CZ<*@VCZ<*@V=\xc8<*@V=\xc8;N@V=\xc8;N?\xff=\xc8;N?\xff<2;N?\xff<2;;?\xff<2;;>\x1c<2;;>\x1c?\x14;;>\x1c?\x14=\xef>\x1c?\x14=\xef>\xdc?\x14=\xef>\xdc>\x95=\xef>\xdc>\x95=a>\xdc>\x95=a<v>\x95=a<v<~=a<v<~<Q<v<~<Q?Q<~<Q?Q?8<Q?Q?8@\xee?]<?:\xdb:&<?:\xdb:&=\xd5:\xdb:&=\xd5>\x07:&=\xd5>\x07@\xe6=\

In [121]:
snap.registers['force_sync'].write_int(2) 
snap.registers['force_sync'].write_int(0) 

In [117]:
gbe1.get_arp_details()

[Mac(FF:FF:FF:FF:FF:FF),
 Mac(F4:52:14:16:24:A0),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),


In [119]:
gbe1.get_gbe_core_details()

{'ip_prefix': '192.168.2',
 'ip': IpAddress(192.168.2.100),
 'subnet_mask': IpAddress(255.255.255.0),
 'mac': Mac(00:08:0B:C4:17:01),
 'gateway_ip': IpAddress(0.0.0.1),
 'fabric_port': 4000,
 'fabric_en': 1,
 'multicast': {'base_ip': IpAddress(255.255.255.255),
  'ip_mask': IpAddress(255.255.255.255),
  'rx_ips': []}}

In [123]:
snap.registers['ip1'].read_uint()

3232236033

In [137]:
# configure gbe0
# src
gbe0_src_mac  = "00:08:0b:c4:17:00"
gbe0_src_ip   = "192.168.2.101"
gbe0_src_port = 4001
gbe0.configure_core(gbe0_src_mac, gbe0_src_ip, gbe0_src_port)
# dst
gbe0_dst_mac  = 0xf452141624a0
# write register requires a int vaule, but set_single_arp_entry requires a string
gbe0_dst_ip   = 192*(2**24) + 168*(2**16) + 2*(2**8) + 1
gbe0_dst_ip_str='192.168.2.1'
gbe0.set_single_arp_entry(gbe0_dst_ip_str,gbe0_dst_mac)
gbe0.fabric_enable()

# configure gbe1
# src
gbe1_src_mac  = "00:08:0b:c4:17:01"
gbe1_src_ip   = "192.168.2.100"
gbe1_src_port = 4000
gbe1.configure_core(gbe1_src_mac, gbe1_src_ip, gbe1_src_port)
# dst
gbe1_dst_mac  = 0xf452141624a0
# write register requires a int vaule, but set_single_arp_entry requires a string
gbe1_dst_ip   = 192*(2**24) + 168*(2**16) + 2*(2**8) + 1
gbe1_dst_ip_str='192.168.2.1'
gbe1.set_single_arp_entry(gbe1_dst_ip_str,gbe1_dst_mac)
gbe1.fabric_enable()

In [135]:
gbe0.get_gbe_core_details()

{'ip_prefix': '192.168.2',
 'ip': IpAddress(192.168.2.101),
 'subnet_mask': IpAddress(255.255.255.0),
 'mac': Mac(00:08:0B:C4:17:02),
 'gateway_ip': IpAddress(0.0.0.1),
 'fabric_port': 4000,
 'fabric_en': 1,
 'multicast': {'base_ip': IpAddress(255.255.255.255),
  'ip_mask': IpAddress(255.255.255.255),
  'rx_ips': []}}

In [136]:
gbe0.get_arp_details()

[Mac(FF:FF:FF:FF:FF:FF),
 Mac(F4:52:14:16:24:A0),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
 Mac(FF:FF:FF:FF:FF:FF),
